# day 7

https://adventofcode.com/7/day/7

In [ ]:
import logging
import logging.config
import os

import yaml

In [ ]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [ ]:
FNAME = os.path.join('data', 'day07.txt')

LOGGER = logging.getLogger('day07')

## part 1

### problem statement:

#### loading data

In [ ]:
test_data = """.......S.......
...............
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
..............."""

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read()

In [ ]:
import networkx as nx

def find_char_locs(char: str, data: str) -> list[complex]:
    return [row + 1j * col
            for (row, line) in enumerate(data.splitlines())
            for (col, c) in enumerate(line)
            if c == char]

def build_graph(origin_node: complex, node_list: list[complex]) -> nx.DiGraph:
    g = nx.DiGraph()
    g.add_nodes_from(node_list)
    g.add_node(origin_node)

    # edges come in two flavors: the highest ^ under S is connected to S; otherwise
    # you can be connected from ^ to ^ if it is (1) above you (2) to the left or right
    # and (3) below your next neighbor up.
    # lookup map will help
    col_to_node_map = {}
    for node in node_list:
        col = node.imag
        try:
            col_to_node_map[col].append(node)
        except KeyError:
            col_to_node_map[col] = [node]

    # S edge
    s_col = origin_node.imag
    highest_s_col_node = min(col_to_node_map[s_col], key=lambda x: x.real)
    g.add_edge(highest_s_col_node, origin_node)

    # neighbor edges
    for col in sorted(col_to_node_map.keys()):
        left_nodes = col_to_node_map[col - 1] if col - 1 in col_to_node_map else []
        right_nodes = col_to_node_map[col + 1] if col + 1 in col_to_node_map else []

        for node in col_to_node_map[col]:
            # you must be one row above the node _and_ one row _below_ the next node above
            # in this column
            try:
                low_bound = max(n.real for n in col_to_node_map[col] if n.real < node.real) + 1
            except:
                low_bound = 0
            high_bound = node.real - 1

            for neighbor in left_nodes + right_nodes:
                if low_bound <= neighbor.real <= high_bound:
                    g.add_edge(node, neighbor)
    return g

In [ ]:
node_list = find_char_locs(char='^', data=test_data)
origin_node = find_char_locs(char='S', data=test_data)[0]
graph = build_graph(origin_node=origin_node, node_list=node_list)
for edge in graph.edges():
    print(edge)

In [ ]:
"""
             11111
   012345678901234
 0 .......S.......
 1 ...............
 2 .......^.......
 3 ...............
 4 ......^.^......
 5 ...............
 6 .....^.^.^.....
 7 ...............
 8 ....^.^...^....
 9 ...............
10 ...^.^...^.^...
11 ...............
12 ..^...^.....^..
13 ...............
14 .^.^.^.^.^...^.
15 ...............

2,7 > 0,7
4,6 > 2,7
4,8 > 2,7
6,5 > 4,6
6,7 > 4,6
6,7 > 4,8
6,9 > 4,8
"""

In [ ]:
def get_edge_nodes(data: str) -> list[complex]:
    n_row = len(data.splitlines())
    n_col = len(data.splitlines()[0])
    return [n_row + 1j * col for col in range(n_col)]

In [ ]:
# get_edge_nodes(data=test_data)

In [ ]:
def parse_raw_data(data: str, with_edge_nodes: bool = False) -> tuple[complex, nx.DiGraph]:
    node_list = find_char_locs(char='^', data=data)
    if with_edge_nodes:
        # add artificial nodes just off the bottom of the map
        node_list += get_edge_nodes(data=data)
    origin_node = find_char_locs(char='S', data=data)[0]
    graph = build_graph(origin_node=origin_node, node_list=node_list)
    return origin_node, graph

In [ ]:
parse_raw_data(test_data)

#### function def

In [ ]:
# node_list = find_char_locs(char='^', data=test_data)
# origin_node = find_char_locs(char='S', data=test_data)[0]
# graph = build_graph(origin_node=origin_node, node_list=node_list)
# nx.ancestors(G=graph, source=origin_node)

In [ ]:
def q_1(data):
    origin_node, graph = parse_raw_data(data=data)
    return len(nx.ancestors(G=graph, source=origin_node))

#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_data) == 21
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(load_data())

## part 2

### problem statement:

#### function def

In [ ]:
def add_num_paths(graph: nx.DiGraph, origin_node: complex) -> None:
    nx.set_node_attributes(G=graph, values=0, name='paths')
    nx.set_node_attributes(G=graph, values={origin_node: {'paths': 1}})
    set_to_update = set(graph.predecessors(origin_node))
    while set_to_update:
        new_set_to_update = set()
        for node in set_to_update:
            paths_val = sum(graph.nodes[n_up]['paths'] for n_up in graph.neighbors(node))
            nx.set_node_attributes(G=graph, values={node: {'paths': paths_val}})
            new_set_to_update.update(graph.predecessors(node))
        set_to_update = new_set_to_update
    return

In [ ]:
# origin_node, graph = parse_raw_data(data=test_data, with_edge_nodes=True)
# add_num_paths(graph=graph, origin_node=origin_node)
#
# for (node, attr) in graph.nodes(data=True):
#     print(f'{node}: {attr["paths"]}')

In [ ]:
# sum(1 for _ in nx.all_simple_paths(G=graph, source=edge_nodes[2], target=origin_node))

In [ ]:
def q_2(data):
    origin_node, graph = parse_raw_data(data=data, with_edge_nodes=True)
    add_num_paths(graph=graph, origin_node=origin_node)
    edge_nodes = get_edge_nodes(data=data)
    return sum(graph.nodes[en]['paths'] for en in edge_nodes)

In [ ]:
# data = load_data()
# origin_node, graph = parse_raw_data(data=data, with_edge_nodes=True)
# add_num_paths(graph=graph, origin_node=origin_node)
# for (node, attr) in graph.nodes(data=True):
#     print(f"{node}: {attr['paths']}")

#### tests

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    assert q_2(test_data) == 40
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_2()

#### answer

In [ ]:
q_2(load_data())

fin